# Day 20: Federated Personalization

**Adapting Global Models to Individual Clients**

## Overview
- **Problem**: Non-IID data causes global model to underperform for some clients
- **Solution**: Personalize model for each client
- **Approach**: Fine-tune global model on local data

## What You'll Learn
1. **Why Personalize?**: Non-IID data heterogeneity
2. **Fine-tuning**: Local adaptation of global model
3. **Meta-Learning**: MAML for FL personalization
4. **Trade-offs**: Personalization vs generalization

---

## 1. The Personalization Problem

**Scenario:**
- Global model trained on ALL clients (FedAvg)
- Client A: North American bank (different fraud patterns)
- Client B: European bank (different fraud patterns)
- **Issue**: Global model averages over all patterns → suboptimal for each

**Solution:**
- Start with global model
- Fine-tune on local data
- Best of both worlds: global knowledge + local adaptation

## 2. Fine-Tuning Approach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("""

PERSONALIZATION VIA FINE-TUNING:

┌─────────────────────────────────────────────────────────┐
│                 FEDERATED LEARNING                      │
│  • Train global model across all clients                 │
│  • Global model: w_global                               │
└─────────────────────────────────────────────────────────┘
                         │
                         ▼
┌─────────────────────────────────────────────────────────┐
│              PERSONALIZATION PHASE                      │
│  For each client k:                                     │
│    1. Start with global model: w_k = w_global           │
│    2. Fine-tune on local data: w_k ← w_k - η ∇L_k(w_k)   │
│    3. Use personalized model for inference              │
└─────────────────────────────────────────────────────────┘

Key Insight:
  Global model provides GOOD INITIALIZATION
  Local fine-tuning adapts to CLIENT'S DATA DISTRIBUTION

""")

## 3. Simulation: Global vs Personalized

In [ ]:
# Simulate FL with personalization
np.random.seed(42)

n_clients = 10
n_rounds = 30

# Generate client data with non-IID distributions
client_data = []
for i in range(n_clients):
    # Each client has different "true" model (non-IID)
    true_weights = np.random.randn(5) + i * 0.2  # Shift by client ID
    X = np.random.randn(100, 5)
    noise = np.random.randn(100) * 0.1
    y = (X @ true_weights + noise > 0).astype(int)
    client_data.append((X, y, true_weights))

# Train global model (simplified FedAvg)
global_weights = np.zeros(5)
global_accuracy = []

for round in range(n_rounds):
    # Client updates
    updates = []
    for X, y, _ in client_data:
        # Gradient step
        preds = (X @ global_weights > 0).astype(int)
        error = y - preds
        grad = -X.T @ error / len(y)
        updates.append(grad)
    
    # FedAvg aggregation
    avg_grad = np.mean(updates, axis=0)
    global_weights -= 0.1 * avg_grad
    
    # Evaluate global model
    accs = []
    for X, y, _ in client_data:
        preds = (X @ global_weights > 0).astype(int)
        acc = np.mean(preds == y)
        accs.append(acc)
    global_accuracy.append(np.mean(accs))

# Personalize: Fine-tune global model on each client
personalized_accuracies = []

for X, y, true_w in client_data:
    # Start with global weights
    personal_weights = global_weights.copy()
    
    # Fine-tune locally
    for epoch in range(10):
        preds = (X @ personal_weights > 0).astype(int)
        error = y - preds
        grad = -X.T @ error / len(y)
        personal_weights -= 0.05 * grad
    
    # Evaluate
    preds = (X @ personal_weights > 0).astype(int)
    acc = np.mean(preds == y)
    personalized_accuracies.append(acc)

# Compare
print("\n" + "="*60)
print("GLOBAL vs PERSONALIZED MODEL PERFORMANCE")
print("="*60)
for i in range(n_clients):
    global_acc = global_accuracy[-1]
    pers_acc = personalized_accuracies[i]
    improvement = pers_acc - global_acc
    print(f"Client {i}: Global={global_acc:.3f}, Personalized={pers_acc:.3f}, "
          f"Improvement={improvement:+.3f}")

print(f"\nAverage global accuracy: {np.mean(global_accuracy):.3f}")
print(f"Average personalized accuracy: {np.mean(personalized_accuracies):.3f}")
print(f"Average improvement: {np.mean(personalized_accuracies) - global_accuracy[-1]:+.3f}")

## 4. Meta-Learning: MAML for FL

In [ ]:
print("""

MAML: MODEL-AGNOSTIC META-LEARNING

Idea: Learn model that can be quickly adapted to new tasks

In FL Context:
  • Each client is a "task" (different data distribution)
  • Learn initialization that adapts quickly to ANY client
  • Better than simple FedAvg for personalization

Algorithm:
  1. Sample batch of clients
  2. For each client:
     a. Compute adaptation: θ'_k = θ - α∇L_k(θ)
     b. Compute meta-loss: L_k(θ'_k) on test data
  3. Meta-update: θ ← θ - β∇(Σ L_k(θ'_k))

Benefits:
  • Learns GOOD INITIALIZATION for all clients
  • Personalization requires only 1-5 gradient steps
  • Outperforms FedAvg fine-tuning

Challenge:
  • Computationally expensive (second-order gradients)
  • More complex implementation

""")

## 5. Personalization Strategies

In [ ]:
strategies_df = pd.DataFrame({
    'Strategy': [
        'No Personalization',
        'Fine-Tuning',
        'MAML',
        'Per-Layer LR',
        'Adapter Layers',
    ],
    'Description': [
        'Use global model as-is',
        'Fine-tune all weights locally',
        'Meta-learn initialization',
        'Different LR per layer (last layers adapt more)',
        'Add small client-specific layers'
    ],
    'Pros': [
        '✅ Simple, no extra cost',
        '✅ Easy to implement',
        '✅ Best performance, fast adaptation',
        '✅ Good balance',
        '✅ Parameter-efficient'
    ],
    'Cons': [
        '❌ Suboptimal for non-IID',
        '❌ Can overfit to local data',
        '❌ Expensive (2nd order)',
        '❌ Requires tuning',
        '❌ More complex architecture'
    ],
})

print("\n" + "="*70)
print("PERSONALIZATION STRATEGIES COMPARISON")
print("="*70)
print(strategies_df.to_string(index=False))

## 6. Summary

### Federated Personalization Summary:

**Problem:**
- Non-IID data causes global model to underperform
- Clients have different data distributions
- One-size-fits-all doesn't work

**Solutions:**

1. **Fine-Tuning** (Most common):
   - Train global model (FedAvg)
   - Fine-tune on local data
   - 5-10 local epochs

2. **MAML** (Best performance):
   - Meta-learn initialization
   - Adapts in 1-5 steps
   - Computationally expensive

3. **Adapter Layers** (Parameter-efficient):
   - Freeze global model
   - Train small client-specific layers
   - Fewer parameters to update

**When to Personalize:**
- ✅ Non-IID data (heterogeneous clients)
- ✅ Client has sufficient local data (>100 samples)
- ✅ Inference latency acceptable

**When NOT to Personalize:**
- ❌ IID data (global model is fine)
- ❌ Very little local data (<50 samples)
- ❌ Strict latency constraints

### Next Steps:
→ **Day 22**: Differential Privacy (privacy guarantees)

---

**📁 Project Location**: `02_federated_learning_foundations/personalized_fl_fraud/`